In [19]:
# hgs_ttp.py
# Hybrid Genetic Search for Bi-objective TTP (GECCO rules)
# - CEIL_2D distances, linear speed drop with carried weight
# - Objectives: f1 = time, f2 = -profit  (minimize both)
# - NSGA-II selection + external ND archive
# - 2-opt (tour) + 1-flip (items) local search
# - Saves final non-dominated (time, profit) to nd_archive.csv and nd_archive.f

import math, random, re, time, csv, os
from typing import List, Tuple, Dict, Any
import time

# =========================
# Problem loader
# =========================

def load_ttp(path: str) -> Dict[str, Any]:
    with open(path, "r", encoding="utf-8") as f:
        lines = [l.rstrip("\n") for l in f]

    num_cities = None
    num_items = None
    coords: List[List[float]] = []
    profits: List[float] = []
    weights: List[float] = []
    city_of_item: List[int] = []
    min_speed = None
    max_speed = None
    max_weight = None
    renting_ratio = None  # loaded, but unused in bi-objective

    i = 0
    while i < len(lines):
        line = lines[i]

        if "DIMENSION" in line:
            num_cities = int(line.split(":")[1].strip())
            coords = [[0.0, 0.0] for _ in range(num_cities)]

        elif "NUMBER OF ITEMS" in line:
            num_items = int(line.split(":")[1].strip())
            profits = [0.0] * num_items
            weights = [0.0] * num_items
            city_of_item = [0] * num_items

        elif "RENTING RATIO" in line:
            renting_ratio = float(line.split(":")[1].strip())

        elif "CAPACITY OF KNAPSACK" in line:
            max_weight = float(line.split(":")[1].strip())

        elif "MIN SPEED" in line:
            min_speed = float(line.split(":")[1].strip())

        elif "MAX SPEED" in line:
            max_speed = float(line.split(":")[1].strip())

        elif "EDGE_WEIGHT_TYPE" in line:
            ew = line.split(":")[1].strip()
            if ew != "CEIL_2D":
                raise RuntimeError("Only CEIL_2D supported")

        elif "NODE_COORD_SECTION" in line:
            for j in range(num_cities):
                i += 1
                parts = re.split(r"\s+", lines[i].strip())
                coords[j][0] = float(parts[1])
                coords[j][1] = float(parts[2])

        elif "ITEMS SECTION" in line:
            for j in range(num_items):
                i += 1
                parts = re.split(r"\s+", lines[i].strip())
                profits[j] = float(parts[1])
                weights[j] = float(parts[2])
                city_of_item[j] = int(parts[3]) - 1

        i += 1

    return {
        "coords": coords,
        "num_cities": num_cities,
        "num_items": num_items,
        "profits": profits,
        "weights": weights,
        "city_of_item": city_of_item,
        "max_weight": max_weight,
        "min_speed": min_speed,
        "max_speed": max_speed,
        "R": renting_ratio,  # loaded for completeness; NOT used in bi-objective
    }

def build_items_at_city(num_cities: int, city_of_item: List[int]) -> List[List[int]]:
    items_at_city: List[List[int]] = [[] for _ in range(num_cities)]
    for j, c in enumerate(city_of_item):
        items_at_city[c].append(j)
    return items_at_city

# CEIL_2D distance
def ceil2d(coords, i, j):
    dx = coords[i][0] - coords[j][0]
    dy = coords[i][1] - coords[j][1]
    return math.ceil(math.sqrt(dx*dx + dy*dy))

def build_candidate_lists(coords, k):
    """
    For each city, return its k nearest neighbours.
    Used to restrict 2-opt neighbourhood.
    """
    n = len(coords)
    cand = [[] for _ in range(n)]
    # compute full distance matrix row by row
    for i in range(n):
        drow = []
        xi, yi = coords[i]
        for j in range(n):
            if i == j:
                continue
            dx = xi - coords[j][0]
            dy = yi - coords[j][1]
            d = dx*dx + dy*dy  # square dist is enough for sorting
            drow.append((d, j))
        drow.sort(key=lambda x: x[0])
        cand[i] = [j for _, j in drow[:k]]
    return cand

# =========================
# Evaluation (bi-objective)
# =========================

def evaluate_ttp(
    pi: List[int],
    z: List[bool],
    coords: List[List[float]],
    items_at_city: List[List[int]],
    weights: List[float],
    profits: List[float],
    max_weight: float,
    min_speed: float,
    max_speed: float,
) -> Tuple[float, float]:
    """
    Bi-objective TTP:
      f1 = total travel time (to minimize)
      f2 = - total profit       (to minimize)
    No renting term in bi-objective. Speed drops linearly with weight.
    """
    n = len(pi)
    if pi[0] != 0:
        raise RuntimeError("Tour must start at city 0")

    time_v = 0.0
    profit_sum = 0.0
    w = 0.0

    for i in range(n):
        city = pi[i]
        # pick items at arrival
        for j in items_at_city[city]:
            if z[j]:
                w += weights[j]
                profit_sum += profits[j]
        if w > max_weight:
            return float("inf"), float("+inf")  # infeasible dominated
        speed = max_speed - (w / max_weight) * (max_speed - min_speed)
        nxt = pi[(i + 1) % n]
        dist = ceil2d(coords, city, nxt)
        time_v += dist / max(speed, 1e-12)

    return time_v, -profit_sum

# =========================
# NSGA-II utilities
# =========================

def nondominated_sort(objs: List[Tuple[float, float]]) -> List[List[int]]:
    N = len(objs)
    S = [[] for _ in range(N)]
    n = [0] * N
    ranks = [0] * N
    fronts: List[List[int]] = [[]]

    def dom(a,b):
        return (objs[a][0] <= objs[b][0] and objs[a][1] <= objs[b][1]) and (objs[a] != objs[b])

    for p in range(N):
        for q in range(N):
            if p==q: continue
            if dom(p,q): S[p].append(q)
            elif dom(q,p): n[p]+=1
        if n[p]==0:
            ranks[p]=0
            fronts[0].append(p)
    i=0
    while fronts[i]:
        nxt=[]
        for p in fronts[i]:
            for q in S[p]:
                n[q]-=1
                if n[q]==0:
                    ranks[q]=i+1
                    nxt.append(q)
        i+=1
        fronts.append(nxt)
    if not fronts[-1]: fronts.pop()
    return fronts

def crowding_distance(front: List[int], objs: List[Tuple[float, float]]) -> Dict[int, float]:
    if not front:
        return {}
    L = len(front)
    dist = {idx: 0.0 for idx in front}
    for m in range(2):
        fs = sorted(front, key=lambda i: objs[i][m])
        dist[fs[0]] = float("inf")
        dist[fs[-1]] = float("inf")
        min_m = objs[fs[0]][m]; max_m = objs[fs[-1]][m]
        denom = (max_m - min_m) if max_m>min_m else 1.0
        for k in range(1, L-1):
            prev = objs[fs[k-1]][m]; nxt = objs[fs[k+1]][m]
            dist[fs[k]] += (nxt - prev)/denom
    return dist


def compute_hv(objs, ideal, nadir):
    # objs in (time, -profit), both minimized
    t_ideal, m_ideal = ideal
    t_nadir, m_nadir = nadir
    if t_nadir <= t_ideal or m_nadir <= m_ideal:
        return 0.0
    pts=[]
    for (t,m) in objs:
        if not (math.isfinite(t) and math.isfinite(m)): continue
        t = min(max(t, t_ideal), t_nadir)
        m = min(max(m, m_ideal), m_nadir)
        if t < t_nadir and m < m_nadir:
            pts.append((t,m))
    if not pts: return 0.0
    pts.sort(key=lambda x: x[0])
    env=[]; best_m=float('inf')
    for (t,m) in pts:
        if m<best_m:
            best_m=m; env.append((t,best_m))
    area=0.0; prev_t=t_nadir
    for t,m in reversed(env):
        w = max(0.0, prev_t - t)
        h = max(0.0, m_nadir - m)
        area += w*h
        prev_t = t
    total = (t_nadir - t_ideal) * (m_nadir - m_ideal)
    if total <= 0: return 0.0
    hv = area/total
    return 0.0 if hv<0 else (1.0 if hv>1 else hv)

# =========================
# Representation & operators
# =========================

class Individual:
    __slots__=("pi","z","obj")
    def __init__(self, pi, z, obj):
        self.pi=pi; self.z=z; self.obj=obj  # (time, -profit)

def random_tour(nc: int) -> List[int]:
    perm=list(range(nc))
    tail=perm[1:]; random.shuffle(tail)
    return [0]+tail

def ox_crossover(a: List[int], b: List[int]) -> List[int]:
    assert a[0]==0 and b[0]==0
    A=a[1:]; B=b[1:]; n1=len(A)
    i,j=sorted(random.sample(range(n1),2))
    mid=A[i:j+1]
    rem=[x for x in B if x not in mid]
    return [0]+rem[:i]+mid+rem[i:]

def mutate_tour_swap(pi: List[int], p):
    if random.random()<p:
        n=len(pi); i,j=random.sample(range(1,n),2)
        pi[i],pi[j]=pi[j],pi[i]

def inherit_items(za: List[bool], zb: List[bool]) -> List[bool]:
    n=len(za); c=[False]*n
    for i in range(n):
        c[i] = za[i] if za[i]==zb[i] else (random.random()<0.5)
    return c

def greedy_density(weights, profits, maxW) -> List[bool]:
    idx=list(range(len(weights)))
    idx.sort(key=lambda j: profits[j]/(weights[j]+1e-9), reverse=True)
    z=[False]*len(weights); W=0.0
    for j in idx:
        w=weights[j]
        if W+w <= maxW:
            z[j]=True; W+=w
    return z

# =========================
# Local search
# =========================

def two_opt_once(pi, eval_fn, z, coords, items_at, weights, profits,
                 maxW, vmin, vmax, cand_lists, max_checks):
    """
    Candidate-restricted 2-opt:
      - For each edge (a,b) in tour
      - For each candidate c in cand_lists[a]  (typically k=10–20)
      - Determine d = successor of c
      - Compute pure-distance delta
      - Only if distance improves, run full TTP eval
    """
    n = len(pi)
    base_t, _ = eval_fn(pi, z, coords, items_at, weights, profits, maxW, vmin, vmax)
    checks = 0

    # map city -> index in pi
    pos = [0] * n
    for idx, city in enumerate(pi):
        pos[city] = idx

    for idx_a in range(n):
        a = pi[idx_a]
        idx_b = (idx_a + 1) % n
        b = pi[idx_b]
        dist_ab = ceil2d(coords, a, b)

        for c in cand_lists[a]:
            idx_c = pos[c]
            idx_d = (idx_c + 1) % n
            d = pi[idx_d]

            if c == a or c == b or d == a:
                continue

            dist_cd = ceil2d(coords, c, d)
            dist_ac = ceil2d(coords, a, c)
            dist_bd = ceil2d(coords, b, d)

            # Pure distance delta
            delta = (dist_ac + dist_bd) - (dist_ab + dist_cd)
            if delta >= 0:
                continue  # reject non-improving candidates

            # Now test full TTP objective
            checks += 1
            if checks > max_checks:
                return False

            # Perform 2-opt excl 0 (ensure correct slice direction)
            i = min(idx_a + 1, idx_c)
            k = max(idx_a + 1, idx_c)

            # forbid reversing through index 0
            if i == 0 or k == 0:
                continue

            new_pi = pi[:i] + pi[i:k+1][::-1] + pi[k+1:]


            t_new, _ = eval_fn(new_pi, z, coords, items_at, weights, profits,
                               maxW, vmin, vmax)
            if math.isfinite(t_new) and t_new + 1e-12 < base_t:
                pi[:] = new_pi
                return True

    return False

def item_1flip_LS(z, pi, eval_fn, coords, items_at, weights, profits, maxW, vmin, vmax, alpha, norm_bounds):
    (tmin,tmax),(mmin,mmax)=norm_bounds
    def score(t,m):
        tn=0 if tmax==tmin else (t-tmin)/(tmax-tmin)
        mn=0 if mmax==mmin else (m-mmin)/(mmax-mmin)
        return alpha*tn + (1-alpha)*mn
    bt,bm = eval_fn(pi,z,coords,items_at,weights,profits,maxW,vmin,vmax)
    if bt==float("inf"):  # infeasible → drop worst density until feasible
        picked=[j for j,b in enumerate(z) if b]
        picked.sort(key=lambda jj: profits[jj]/(weights[jj]+1e-9))
        for jj in picked:
            z[jj]=False
            t,m=eval_fn(pi,z,coords,items_at,weights,profits,maxW,vmin,vmax)
            if t!=float("inf"): bt, bm = t, m; break
        else:
            return False
    bs=score(bt,bm)
    order=list(range(len(z))); random.shuffle(order)
    for j in order:
        z[j]=not z[j]
        t,m=eval_fn(pi,z,coords,items_at,weights,profits,maxW,vmin,vmax)
        if t==float("inf"):
            z[j]=not z[j]; continue
        s=score(t,m)
        if s+1e-12 < bs:
            return True
        z[j]=not z[j]
    return False

def local_search(pi, z, eval_fn, coords, items_at, weights, profits, maxW, vmin, vmax, alpha, norm_bounds, max_iters, max_2opt_checks, cand_lists):



    for _ in range(max_iters):
        imp=False
        imp |= two_opt_once(pi, eval_fn, z, coords, items_at, weights, profits, maxW, vmin, vmax, cand_lists, max_2opt_checks)


        imp |= item_1flip_LS(z,pi,eval_fn,coords,items_at,weights,profits,maxW,vmin,vmax,alpha,norm_bounds)
        if not imp: break

# =========================
# Archive ops
# =========================

def pareto_insert(archive: List[Individual], cand: Individual) -> None:
    if not (math.isfinite(cand.obj[0]) and math.isfinite(cand.obj[1])): return
    # dominated by existing?
    for s in archive:
        if (s.obj[0] <= cand.obj[0] and s.obj[1] <= cand.obj[1]) and (s.obj != cand.obj):
            return
    # keep only those not dominated by cand
    nd=[]
    for s in archive:
        if (cand.obj[0] <= s.obj[0] and cand.obj[1] <= s.obj[1]) and (cand.obj != s.obj):
            continue
        nd.append(s)
    nd.append(cand)
    archive[:] = nd

# =========================
# HGS main
# =========================

def make_individual(pi, z, eval_fn, prob):
    obj = eval_fn(pi,z,prob["coords"],prob["items_at"],prob["weights"],prob["profits"],
                  prob["max_weight"],prob["min_speed"],prob["max_speed"])
    return Individual(pi,z,obj)

def run_hgs(prob: Dict[str, Any]):
    random.seed(42)
    # === CENTRAL HYPERPARAMETER BLOCK ===

    PRINT_EVERY = 1     # reduce or increase depending on variables being tested

    # Population structure
    POP_SIZE = 50
    OFFSPRING = 50
    GENS = 100

    # Tournament size (new)
    TOUR_K = 3           # selection pressure; 2 = minimal, >3 = more greedy

    # Archive control
    MAX_ARCHIVE = 4000  # Maximum number of non-dominated solutions stored in the external archive

    # Mutation
    MUT_TOUR_P = 0.01    # Probability the tour receives a swap mutation (one swap of two cities).
    MUT_ITEM_P = 0.01    # Probability that one random item bit flips after crossover.

    # Local search
    LS_MAX_ITERS = 100    # Max number of LS cycles per child. One LS iteration = Try one improving 2-opt move (tour), then try one improving 1-flip (items).
    C_2OPT = 8            # MAX_2OPT_CHECKS = C_2OPT * n_cities
    LS_ALPHA_SCHEDULE = [1.0, 1.0, 1.0, 0.5, 0.0]   # cycling bias toward time→balanced→profit
    CANDIDATE_K = 15   # local city search candidates

    # Bounds behaviour
    EARLY_GLOBAL_BOUNDS_GENS = 200

    # Timing
    MAX_GEN_SEC = 5000

    # Seeding ratios
    INIT_EMPTY_RATIO = 0.2      # fraction of POP
    INIT_SPEED_AWARE_RATIO = 0.33 
    INIT_PROFIT_HEAVY_RATIO = 0.33
    # remainder = random feasible

    RANDOM_PACK_PROB = 0.10



    # HV box commonly used for a280-n279 (time, -profit)
    IDEAL=(2613.0, -42036.0)
    NADIR=(5444.0, -0.0)

    prob["items_at"] = build_items_at_city(prob["num_cities"], prob["city_of_item"])

    # init population: empty, density-greedy, random feasible
    pop: List[Individual]=[]
    nC = prob["num_cities"]; nI = prob["num_items"]
    cand_lists = build_candidate_lists(prob["coords"], CANDIDATE_K)

    def seed_empty_fast():
        pi = random_tour(nC)
        for _ in range(4):
            two_opt_once(pi, evaluate_ttp, [False]*nI,
                        prob["coords"], prob["items_at"], prob["weights"], prob["profits"],
                        prob["max_weight"], prob["min_speed"], prob["max_speed"],
                        cand_lists,
                        C_2OPT * nC)


        z = [False]*nI
        return make_individual(pi,z,evaluate_ttp,prob)
    
    def seed_speed_aware():
        pi = random_tour(nC)
        # compute remaining distance from each city along the tour
        rem = [0.0] * nC
        acc = 0.0
        for i in range(nC-1, -1, -1):
            j = (i+1) % nC
            acc += ceil2d(prob["coords"], pi[i], pi[j])
            rem[i] = acc
        pos = [0] * nC
        for idx, city in enumerate(pi):
            pos[city] = idx

        cand = []
        vmax = prob["max_speed"]


        for city in range(nC):
            for j in prob["items_at"][city]:
                w = prob["weights"][j]
                p = prob["profits"][j]
                rd = rem[pos[city]]
                penalty = rd * w / vmax
                score = p - penalty
                cand.append((score, j, w))

        cand.sort(key=lambda x: x[0], reverse=True)
        z = [False] * nI
        W = 0.0
        for score, j, w in cand:
            if W + w <= prob["max_weight"]:
                z[j] = True
                W += w
        return make_individual(pi, z, evaluate_ttp, prob)


    def seed_density_greedy():
        pi = random_tour(nC)
        z = greedy_density(prob["weights"], prob["profits"], prob["max_weight"])
        return make_individual(pi,z,evaluate_ttp,prob)
    

    seeds = []

    num_empty   = max(2, int(POP_SIZE * INIT_EMPTY_RATIO))
    num_speed   = max(2, int(POP_SIZE * INIT_SPEED_AWARE_RATIO))
    num_greedy  = max(2, int(POP_SIZE * INIT_PROFIT_HEAVY_RATIO))

    seeds += [seed_empty_fast() for _ in range(num_empty)]
    seeds += [seed_speed_aware() for _ in range(num_speed)]
    seeds += [seed_density_greedy() for _ in range(num_greedy)]


    while len(seeds) < POP_SIZE:
        pi = random_tour(nC)
        z=[False]*nI; W=0.0
        for j in random.sample(range(nI), nI):
            w=prob["weights"][j]
            if random.random() < RANDOM_PACK_PROB and W+w <= prob["max_weight"]:
                z[j]=True; W+=w
        seeds.append(make_individual(pi,z,evaluate_ttp,prob))
    pop = seeds[:POP_SIZE]

    archive: List[Individual]=[]
    for ind in pop: pareto_insert(archive, ind)

    last_time = time.time()
    best_prev = float("inf")

    for g in range(GENS):
        objs=[ind.obj for ind in pop]
        fronts = nondominated_sort(objs)
        ranks=[None]*len(pop)
        for r,F in enumerate(fronts):
            for idx in F: ranks[idx]=r
        distances={}
        for F in fronts:
            distances.update(crowding_distance(F, objs))

        children: List[Individual]=[]
        for _ in range(OFFSPRING):
            # tournament select (rank, then crowding)

            def tournament_select():
                cand = random.sample(range(len(pop)), TOUR_K)
                cand_sorted = sorted(
                    cand,
                    key=lambda idx: (ranks[idx], -distances.get(idx, 0.0))
                )
                return pop[cand_sorted[0]]

            p1 = tournament_select()
            p2 = tournament_select()


            child_pi = ox_crossover(p1.pi, p2.pi)
            child_z = inherit_items(p1.z, p2.z)
            mutate_tour_swap(child_pi, p=MUT_TOUR_P)
            if random.random() < MUT_ITEM_P:
                j = random.randrange(len(child_z))
                child_z[j] = not child_z[j]


            # LS normalization bounds
            if g < EARLY_GLOBAL_BOUNDS_GENS:
                # Stable heuristic early bounds
                # (teams used constants or wide heuristic ranges)
                tmin = 2000.0
                tmax = 80000.0
                mmin = -120000.0   # -profit
                mmax = 0.0
            else:
                # Population-based bounds (what teams actually used)
                ts = [o[0] for o in objs if math.isfinite(o[0])]
                ms = [o[1] for o in objs if math.isfinite(o[1])]
                if not ts: ts = [0.0, 1.0]
                if not ms: ms = [-1.0, 0.0]

                tmin, tmax = min(ts), max(ts)
                mmin, mmax = min(ms), max(ms)

                # avoid zero ranges
                if tmax == tmin: tmax = tmin + 1.0
                if mmax == mmin: mmax = mmin + 1.0

            alpha = LS_ALPHA_SCHEDULE[g % len(LS_ALPHA_SCHEDULE)]
            MAX_2OPT_CHECKS = C_2OPT * nC

            local_search(child_pi, child_z, evaluate_ttp,
                        prob["coords"], prob["items_at"], prob["weights"], prob["profits"],
                        prob["max_weight"], prob["min_speed"], prob["max_speed"],
                        alpha, ((tmin,tmax),(mmin,mmax)),
                        max_iters=LS_MAX_ITERS,
                        max_2opt_checks=MAX_2OPT_CHECKS,
                        cand_lists=cand_lists)




            child = make_individual(child_pi, child_z, evaluate_ttp, prob)
            children.append(child)
            pareto_insert(archive, child)

            if len(archive) > MAX_ARCHIVE:
                objs_arch=[s.obj for s in archive]
                fronts_arch=nondominated_sort(objs_arch)
                new_arch=[]
                for F in fronts_arch:
                    if len(new_arch)+len(F) <= MAX_ARCHIVE:
                        new_arch += [archive[i] for i in F]
                    else:
                        d=crowding_distance(F, objs_arch)
                        F_sorted=sorted(F, key=lambda i: d.get(i,0.0), reverse=True)
                        need = MAX_ARCHIVE - len(new_arch)
                        new_arch += [archive[i] for i in F_sorted[:need]]
                        break
                archive[:] = new_arch

        # Environmental selection
        combined = pop + children
        c_objs=[ind.obj for ind in combined]
        c_fronts = nondominated_sort(c_objs)
        new_pop=[]
        for F in c_fronts:
            if len(new_pop)+len(F) <= POP_SIZE:
                new_pop += [combined[i] for i in F]
            else:
                d=crowding_distance(F, c_objs)
                F_sorted = sorted(F, key=lambda i: d.get(i,0.0), reverse=True)
                need = POP_SIZE - len(new_pop)
                new_pop += [combined[i] for i in F_sorted[:need]]
                break
        pop = new_pop

        # occasional stagnation kick on best time
        if (g+1)%200==0:
            best_now = min(ind.obj[0] for ind in archive)
            if g>0 and abs(best_now - best_prev) < 1e-9:
                for _ in range(max(1, POP_SIZE//25)):
                    pi=random_tour(nC)
                    z=[False]*nI
                    pop[random.randrange(len(pop))]=make_individual(pi,z,evaluate_ttp,prob)
            best_prev = best_now

        # per-gen wall clock cap
        if time.time()-last_time > MAX_GEN_SEC:
            print(f"[GEN TIME LIMIT HIT] g={g+1}, aborting gen early", flush=True)
            last_time=time.time()
            continue


        # logging
        if (g+1)%PRINT_EVERY==0:
            elapsed=time.time()-last_time; last_time=time.time()
            best_t = min(s.obj[0] for s in archive)
            best_profit = max(-s.obj[1] for s in archive)
            # HV box is for (time, -profit)
            hv = compute_hv([s.obj for s in archive], IDEAL, NADIR)
            print(f"Gen {g+1}: arch={len(archive)} best_time={best_t:.3f} best_profit={best_profit:.0f} HV={hv:.4f} gen_sec={elapsed:.2f}", flush=True)

    final_hv = compute_hv([s.obj for s in archive], IDEAL, NADIR)
    print(f"\nFinal HV: {final_hv:.4f}")

    # sort ND archive by time and return
    archive.sort(key=lambda s: (s.obj[0], s.obj[1]))
    return archive

# =========================
# Main
# =========================

def main():
    # a280-n279 default path
    path_candidates = ["gecco19-thief/src/main/resources/a280-n279.txt"]
    PATH=None
    for p in path_candidates:
        if os.path.exists(p):
            PATH=p; break
    if PATH is None:
        raise RuntimeError("Instance file a280-n279.txt not found.")

    prob = load_ttp(PATH)
    nd = run_hgs(prob)

    # dumps: exact competition format (time, profit)
    with open("nd_archive.csv","w",newline="",encoding="utf-8") as f:
        w=csv.writer(f); w.writerow(["time","profit"])
        for s in nd:
            w.writerow([f"{s.obj[0]:.6f}", f"{-s.obj[1]:.6f}"])

    with open("nd_archive.f","w",encoding="utf-8") as f:
        for s in nd:
            f.write(f"{s.obj[0]:.9f} {-s.obj[1]:.0f}\n")

    print("\nSaved: nd_archive.csv (time, profit), nd_archive.f (time profit)")

if __name__=="__main__":
    # wallclock anchor for per-gen cap
    last_time = time.time()
    main()


Gen 1: arch=7 best_time=22807.000 best_profit=41763 HV=0.0000 gen_sec=209.14
Gen 2: arch=7 best_time=17902.000 best_profit=41763 HV=0.0000 gen_sec=205.43
Gen 3: arch=7 best_time=16019.000 best_profit=41763 HV=0.0000 gen_sec=200.79
Gen 4: arch=7 best_time=16019.000 best_profit=41763 HV=0.0000 gen_sec=96.19
Gen 5: arch=5 best_time=16019.000 best_profit=41763 HV=0.0000 gen_sec=359.41
Gen 6: arch=5 best_time=8482.000 best_profit=41763 HV=0.0000 gen_sec=195.14
Gen 7: arch=5 best_time=6352.000 best_profit=41763 HV=0.0000 gen_sec=180.92
Gen 8: arch=5 best_time=4822.000 best_profit=41763 HV=0.0000 gen_sec=199.31
Gen 9: arch=4 best_time=4822.000 best_profit=41763 HV=0.0000 gen_sec=112.53
Gen 10: arch=9 best_time=4822.000 best_profit=41763 HV=0.0000 gen_sec=359.97
Gen 11: arch=9 best_time=3134.000 best_profit=41763 HV=0.0000 gen_sec=178.39
Gen 12: arch=9 best_time=2910.000 best_profit=41763 HV=0.0000 gen_sec=157.20
Gen 13: arch=9 best_time=2813.000 best_profit=41763 HV=0.0000 gen_sec=162.45
Gen 